<a href="https://colab.research.google.com/github/bw-spotify2/ds/blob/master/Song_Selector_Model_Building.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Initial building of the model.

### Load in the Dataset

In [1]:
import pandas as pd

df = pd.read_csv('/content/drive/My Drive/Lambda/Unit 4/Sprint 4/SpotifyFeatures.csv')
df.sample(10)

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
67389,Hip-Hop,Omar LinX,Keep It Mello (feat. Omar LinX),0Q3SC6kEhxYagDP3bFe5K9,72,0.00134,0.686,243380,0.888,0.000689,C#,0.0736,-3.348,Major,0.0857,142.015,4/4,0.108
71557,Blues,Etta James,Never My Love,78gf2TxprPQbRSknY7bJ8u,28,0.82200,0.624,232667,0.409,0.022200,G#,0.1200,-8.499,Major,0.0354,129.383,4/4,0.334
5980,Alternative,MGMT,Weekend Wars,19K3lUMJmOdeuOBTrbLm19,53,0.09640,0.549,250773,0.859,0.000024,F,0.3220,-3.993,Major,0.0361,77.013,4/4,0.690
38289,Electronic,Justice,Randy (Radio Edit),53NHTlQoXw7cf96cdK35R0,41,0.00261,0.642,193471,0.695,0.000004,D,0.1040,-4.778,Major,0.0264,100.454,4/4,0.276
59104,Opera,Tomaso Albinoni,"Oboe Concerto in G Minor, Op. 9, No. 8: II. Ad...",6V19IMv8mPCxjxiNXdp2Tw,25,0.94400,0.163,135333,0.130,0.950000,A#,0.1360,-19.531,Major,0.0362,92.264,3/4,0.121
42686,Folk,Sufjan Stevens,Lonely Man of Winter - Doveman Mix feat. Melis...,593ZLxBJwoqT9WkkTTI2HA,55,0.87200,0.560,219643,0.663,0.475000,C,0.1810,-10.444,Major,0.0465,111.989,4/4,0.339
54260,Movie,Chorus,Aarti Kijai Hanuman Lala Ki,3uoGVDpgPAl9RP2uqWxgnO,0,0.89700,0.576,287293,0.740,0.000000,C#,0.2600,-8.148,Major,0.0688,92.126,4/4,0.701
176369,Jazz,Minnie Riperton,Les Fleurs,5ZBZSepIdSYbPaTSZLvZBU,48,0.76700,0.437,196682,0.525,0.000042,G#,0.2500,-8.074,Major,0.0497,152.469,4/4,0.478
150296,Pop,j-hope,P.O.P (Piece Of Peace) pt.1,2xYpEh4RtRcHOK5XOG7XQz,63,0.00652,0.547,181113,0.845,0.000002,F#,0.0604,-5.293,Major,0.0866,175.816,4/4,0.670
225473,Rock,Sufjan Stevens,Casimir Pulaski Day,1u1YU1LE0FWHFOHpR2iXua,61,0.90900,0.686,353947,0.172,0.001540,G#,0.2860,-14.979,Major,0.0409,133.899,4/4,0.563


In [2]:
df['mode'] = df['mode'].replace({"Major":1, "Minor":0})
df.head()

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,99373,0.910,0.000,C#,0.3460,-1.828,1,0.0525,166.969,4/4,0.814
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.246,0.590,137373,0.737,0.000,F#,0.1510,-5.559,0,0.0868,174.003,4/4,0.816
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.952,0.663,170267,0.131,0.000,C,0.1030,-13.879,0,0.0362,99.488,5/4,0.368
3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.703,0.240,152427,0.326,0.000,C#,0.0985,-12.178,1,0.0395,171.758,4/4,0.227
4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.950,0.331,82625,0.225,0.123,F,0.2020,-21.150,1,0.0456,140.576,4/4,0.390


In [3]:
df['time_signature'].value_counts()

4/4    200760
3/4     24111
5/4      5238
1/4      2608
0/4         8
Name: time_signature, dtype: int64

In [4]:
df['time_signature'] = df['time_signature'].replace({'4/4':4,'3/4':3,'5/4':5,'1/4':1,'0/4':0})
df.head()

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,99373,0.910,0.000,C#,0.3460,-1.828,1,0.0525,166.969,4,0.814
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.246,0.590,137373,0.737,0.000,F#,0.1510,-5.559,0,0.0868,174.003,4,0.816
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.952,0.663,170267,0.131,0.000,C,0.1030,-13.879,0,0.0362,99.488,5,0.368
3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.703,0.240,152427,0.326,0.000,C#,0.0985,-12.178,1,0.0395,171.758,4,0.227
4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.950,0.331,82625,0.225,0.123,F,0.2020,-21.150,1,0.0456,140.576,4,0.390


In [5]:
df['key'].value_counts()

C     27583
G     26390
D     24077
C#    23201
A     22671
F     20279
B     17661
E     17390
A#    15526
F#    15222
G#    15159
D#     7566
Name: key, dtype: int64

In [6]:
key_dict = {'C':0,'C#':1,'D':2,'D#':3,'E':4,'F':5,'F#':6,'G':7,'G#':8,'A':9,'A#':10,'B':11}
df['key'] = df['key'].replace(key_dict)
df.head()

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,99373,0.910,0.000,1,0.3460,-1.828,1,0.0525,166.969,4,0.814
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.246,0.590,137373,0.737,0.000,6,0.1510,-5.559,0,0.0868,174.003,4,0.816
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.952,0.663,170267,0.131,0.000,0,0.1030,-13.879,0,0.0362,99.488,5,0.368
3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.703,0.240,152427,0.326,0.000,1,0.0985,-12.178,1,0.0395,171.758,4,0.227
4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.950,0.331,82625,0.225,0.123,5,0.2020,-21.150,1,0.0456,140.576,4,0.390


In [7]:
# Connect to the API in case we need to look at it's output

!pip install spotipy --upgrade
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time 

client_id = '14f59f665a094868921bdd76e420d7c0'
client_secret = '5e16ddb0f2ed45a69949330bbd34351a'
username = 'mahoryu'

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

### Define the columns we want to use and make a df with it

In [55]:
# define headers
headers = ['popularity','acousticness','danceability','duration_ms','energy','instrumentalness','liveness','loudness','speechiness','tempo','valence', 'key', 'mode', 'time_signature']
df_new = df[headers]

In [56]:
df_new.head()

,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,key,mode,time_signature
0,0,0.611,0.389,99373,0.910,0.000,0.3460,-1.828,0.0525,166.969,0.814,1,1,4
1,1,0.246,0.590,137373,0.737,0.000,0.1510,-5.559,0.0868,174.003,0.816,6,0,4
2,3,0.952,0.663,170267,0.131,0.000,0.1030,-13.879,0.0362,99.488,0.368,0,0,5
3,0,0.703,0.240,152427,0.326,0.000,0.0985,-12.178,0.0395,171.758,0.227,1,1,4
4,4,0.950,0.331,82625,0.225,0.123,0.2020,-21.150,0.0456,140.576,0.390,5,1,4


In [57]:
df_new.shape

(232725, 14)

### Make the model

In [58]:
# imports 

from keras.models import Model
from keras.layers import Input, Dense
from keras import regularizers
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

In [59]:
# Normalise the data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(df_new)

#### Pickle the Scaler

In [60]:
import pickle

# Dump the trained classifier (autoencoder)  with Pickle
pickle_filename_1 = 'scaler_model.pkl'
pickled_model_1 = open(pickle_filename_1, 'wb')  # Open the file to save as pkl file
pickle.dump(scaler, pickled_model_1)
pickled_model_1.close() # Close the pickle instances

# Loading the saved model
model_pkl_1 = open(pickle_filename_1, 'rb')
scaler_model = pickle.load(model_pkl_1)
print ("Loaded model :: ", scaler_model)  # print to verify

Loaded model ::  MinMaxScaler(copy=True, feature_range=(0, 1))


#### Create actual Model

In [61]:
# set the scaled data to a new df
df_scaled = pd.DataFrame(data_scaled,columns=headers)
df_scaled.head()

,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,key,mode,time_signature
0,0.00,0.613454,0.356292,0.015167,0.910909,0.000000,0.339614,0.900856,0.032070,0.642704,0.814,0.090909,1.0,0.8
1,0.01,0.246988,0.571934,0.022029,0.737732,0.000000,0.142710,0.834469,0.068374,0.675801,0.816,0.545455,0.0,0.8
2,0.03,0.955823,0.650252,0.027969,0.131113,0.000000,0.094241,0.686429,0.014818,0.325182,0.368,0.000000,0.0,1.0
3,0.00,0.705823,0.196438,0.024747,0.326313,0.000000,0.089697,0.716695,0.018311,0.665238,0.227,0.090909,1.0,0.8
4,0.04,0.953815,0.294067,0.012142,0.225209,0.123123,0.194208,0.557054,0.024767,0.518516,0.390,0.454545,1.0,0.8


In [65]:
# create the actual autoencoder

# set the input dimentions to what is in the df_new
input_dim = df_new.shape[1]  # 14

# Number of neurons in each Layer [14, 7, 3, ...] of encoders
input_layer = Input(shape=(input_dim, ))
encoder_layer_1 = Dense(8, activation="relu", activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoder_layer_2 = Dense(3, activation="relu")(encoder_layer_1)

# Crear encoder model
encoder = Model(inputs=input_layer, outputs=encoder_layer_2)

decoder_1 = Dense(8, activation='relu')(encoder_layer_2)
decoder_output = Dense(input_dim, activation = 'sigmoid')(decoder_1)

autoencoder = Model(input_layer, decoder_output)

autoencoder.summary()

Model: "model_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 14)                0         
_________________________________________________________________
dense_25 (Dense)             (None, 8)                 120       
_________________________________________________________________
dense_26 (Dense)             (None, 3)                 27        
_________________________________________________________________
dense_27 (Dense)             (None, 8)                 32        
_________________________________________________________________
dense_28 (Dense)             (None, 14)                126       
Total params: 305
Trainable params: 305
Non-trainable params: 0
_________________________________________________________________


In [66]:
from tensorflow.keras.callbacks import EarlyStopping

# compile the autoencoder
autoencoder.compile(optimizer='nadam',
                    loss='binary_crossentropy') 
                    
stop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3)

# fit the model
autoencoder.fit(df_scaled, df_scaled,
                epochs=100,
                batch_size=128,
                validation_split=0.2,
                callbacks=[stop]) 

Train on 186180 samples, validate on 46545 samples
Epoch 1/100
186180/186180 [==============================] - 8s 41us/step - loss: 0.5098 - val_loss: 0.4757
Epoch 2/100
186180/186180 [==============================] - 7s 40us/step - loss: 0.4778 - val_loss: 0.4716
Epoch 3/100
186180/186180 [==============================] - 8s 41us/step - loss: 0.4693 - val_loss: 0.4562
Epoch 4/100
186180/186180 [==============================] - 8s 43us/step - loss: 0.4561 - val_loss: 0.4487
Epoch 5/100
186180/186180 [==============================] - 8s 42us/step - loss: 0.4512 - val_loss: 0.4462
Epoch 6/100
186180/186180 [==============================] - 7s 40us/step - loss: 0.4496 - val_loss: 0.4455
Epoch 7/100
186180/186180 [==============================] - 8s 42us/step - loss: 0.4486 - val_loss: 0.4449
Epoch 8/100
186180/186180 [==============================] - 8s 42us/step - loss: 0.4479 - val_loss: 0.4445
Epoch 9/100
186180/186180 [==============================] - 8s 42us/step - loss: 0.4

In [67]:
# Use the model to predict the factors which sum up the selected atributes of the songs,
#   by making an encoded_data df.
encoded_data = pd.DataFrame(encoder.predict(data_scaled))
encoded_data.columns = ['x', 'y', 'z']
encoded_data.head()

,x,y,z
0,1.273887,0.832778,0.433250
1,0.280818,0.413931,0.399893
2,0.155448,0.020456,0.424335
3,1.406550,0.740738,0.522780
4,1.234502,0.601306,0.528400


#### Try with less columns

(No go, we end up with a higher loss)

In [50]:
headers = ['acousticness','danceability','energy','instrumentalness','loudness','speechiness','valence', 'key', 'mode']
df_new = df[headers]

In [51]:
# Normalise the data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(df_new)

# set the scaled data to a new df
df_scaled = pd.DataFrame(data_scaled,columns=headers)
df_scaled.head()

,acousticness,danceability,energy,instrumentalness,loudness,speechiness,valence,key,mode
0,0.613454,0.356292,0.910909,0.000000,0.900856,0.032070,0.814,0.090909,1.0
1,0.246988,0.571934,0.737732,0.000000,0.834469,0.068374,0.816,0.545455,0.0
2,0.955823,0.650252,0.131113,0.000000,0.686429,0.014818,0.368,0.000000,0.0
3,0.705823,0.196438,0.326313,0.000000,0.716695,0.018311,0.227,0.090909,1.0
4,0.953815,0.294067,0.225209,0.123123,0.557054,0.024767,0.390,0.454545,1.0


In [53]:
# create the actual autoencoder

# set the input dimentions to what is in the df_new
input_dim = df_new.shape[1]  # 14

# Number of neurons in each Layer [14, 7, 3, ...] of encoders
input_layer = Input(shape=(input_dim, ))
encoder_layer_1 = Dense(6, activation="relu", activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoder_layer_2 = Dense(3, activation="relu")(encoder_layer_1)

# Crear encoder model
encoder = Model(inputs=input_layer, outputs=encoder_layer_2)

decoder_1 = Dense(6, activation='relu')(encoder_layer_2)
decoder_output = Dense(input_dim, activation = 'sigmoid')(decoder_1)

autoencoder = Model(input_layer, decoder_output)

autoencoder.summary()

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 9)                 0         
_________________________________________________________________
dense_17 (Dense)             (None, 6)                 60        
_________________________________________________________________
dense_18 (Dense)             (None, 3)                 21        
_________________________________________________________________
dense_19 (Dense)             (None, 6)                 24        
_________________________________________________________________
dense_20 (Dense)             (None, 9)                 63        
Total params: 168
Trainable params: 168
Non-trainable params: 0
_________________________________________________________________


In [54]:
# compile the autoencoder
autoencoder.compile(optimizer='nadam',
                    loss='binary_crossentropy') 
                    
stop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3)

# fit the model
autoencoder.fit(df_scaled, df_scaled,
                epochs=100,
                batch_size=128,
                validation_split=0.2,
                callbacks=[stop]) 

Train on 186180 samples, validate on 46545 samples
Epoch 1/100
186180/186180 [==============================] - 8s 42us/step - loss: 0.5335 - val_loss: 0.5178
Epoch 2/100
186180/186180 [==============================] - 8s 43us/step - loss: 0.4768 - val_loss: 0.4740
Epoch 3/100
186180/186180 [==============================] - 8s 43us/step - loss: 0.4680 - val_loss: 0.4668
Epoch 4/100
186180/186180 [==============================] - 8s 43us/step - loss: 0.4654 - val_loss: 0.4621
Epoch 5/100
186180/186180 [==============================] - 8s 42us/step - loss: 0.4640 - val_loss: 0.4603
Epoch 6/100
186180/186180 [==============================] - 8s 43us/step - loss: 0.4632 - val_loss: 0.4592
Epoch 7/100
186180/186180 [==============================] - 8s 45us/step - loss: 0.4626 - val_loss: 0.4591
Epoch 8/100
186180/186180 [==============================] - 8s 43us/step - loss: 0.4622 - val_loss: 0.4590
Epoch 9/100
186180/186180 [==============================] - 8s 43us/step - loss: 0.4

#### Other Testing

In [86]:
########
## try resonstructing a song to see accuracy visually.
########
df_input = df_scaled[:1]
df_output = pd.DataFrame(autoencoder.predict(df_input),columns=headers)

In [87]:
# Starting scaled data
df_input.head()

,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,key,mode,time_signature
0,0.0,0.613454,0.356292,0.015167,0.910909,0.0,0.339614,0.900856,0.03207,0.642704,0.814,0.090909,1.0,0.8


In [88]:
# reconstructed scaled data
df_output.head()

,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,key,mode,time_signature
0,0.402415,0.323837,0.568413,0.039386,0.630416,0.004537,0.241853,0.789318,0.097005,0.414251,0.47817,0.107396,1.0,0.777633


In [89]:
# the difference between the two
df_difference = df_input-df_output
df_difference.head()

,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,key,mode,time_signature
0,-0.402415,0.289617,-0.21212,-0.024219,0.280493,-0.004537,0.097761,0.111537,-0.064935,0.228453,0.33583,-0.016487,0.0,0.022367


In [25]:
# plot the x,y coordinates
# encoded_data.plot.scatter('x','y')

#### Pickle the Model

In [26]:
import pickle

# Dump the trained classifier (autoencoder)  with Pickle
pickle_filename = 'autoencoder_model.pkl'
pickled_model = open(pickle_filename, 'wb')  # Open the file to save as pkl file
pickle.dump(autoencoder, pickled_model)
pickled_model.close() # Close the pickle instances

# Loading the saved model
model_pkl = open(pickle_filename, 'rb')
autoencoder_model = pickle.load(model_pkl)
print ("Loaded model :: ", autoencoder_model)  # print to verify

Loaded model ::  <keras.engine.training.Model object at 0x7f35519439b0>


In [28]:
# Get test data

df_input = df_scaled[:1]
encoded_data_test = pd.DataFrame(encoder.predict(df_input))
encoded_data_test.columns = ['x', 'y', 'z']
encoded_data_test.head()

,x,y,z
0,0.869386,0.805411,0.477762


In [29]:
test_x = encoder.predict(df_input)[0][0]
test_y = encoder.predict(df_input)[0][1]
test_z = encoder.predict(df_input)[0][2]

### Get Distance

In [30]:
# example of euclidian distance
from scipy.spatial import distance
a = (1, 2, 3)
b = (4, 5, 6)
dst = distance.euclidean(a, b)

In [31]:
encoded_data.head()

,x,y,z
0,0.869386,0.805411,0.477762
1,0.665960,1.143413,0.350866
2,0.588679,0.669358,0.309636
3,0.709266,0.630260,0.426421
4,0.763386,0.510344,0.486433


In [32]:
encoded_data['track_id'] = df['track_id']
encoded_data.head()

,x,y,z,track_id
0,0.869386,0.805411,0.477762,0BRjO6ga9RKCKjfDqeFgWV
1,0.665960,1.143413,0.350866,0BjC1NfoEOOusryehmNudP
2,0.588679,0.669358,0.309636,0CoSDzoNIKCRs124s9uTVy
3,0.709266,0.630260,0.426421,0Gc6TVm52BwZD07Ki6tIvf
4,0.763386,0.510344,0.486433,0IuslXpMROHdEPvSl1fTQK


In [33]:
# save the encoded data as a .csv file    

encoded_data.to_csv("encoded_data.csv", index=False)

### Test out the pickled model

In [34]:
# test out reading it in.
df1 = pd.read_csv("encoded_data.csv")
df1.head()

,x,y,z,track_id
0,0.869386,0.805411,0.477762,0BRjO6ga9RKCKjfDqeFgWV
1,0.665960,1.143413,0.350866,0BjC1NfoEOOusryehmNudP
2,0.588679,0.669358,0.309636,0CoSDzoNIKCRs124s9uTVy
3,0.709266,0.630260,0.426421,0Gc6TVm52BwZD07Ki6tIvf
4,0.763386,0.510344,0.486433,0IuslXpMROHdEPvSl1fTQK


In [35]:
encoded_data_test[['x','y','z']].values

array([[0.86938643, 0.8054109 , 0.47776186]], dtype=float32)

In [38]:
def get_e_dist(in_df):
  return distance.euclidean(in_df[[0,1,2]], (test_x,test_y,test_z))

In [39]:
encoded_data['e_distance'] = encoded_data.apply(get_e_dist, axis=1)

In [40]:
encoded_data.head()

,x,y,z,track_id,e_distance
0,0.869386,0.805411,0.477762,0BRjO6ga9RKCKjfDqeFgWV,0.000000
1,0.665960,1.143413,0.350866,0BjC1NfoEOOusryehmNudP,0.414404
2,0.588679,0.669358,0.309636,0CoSDzoNIKCRs124s9uTVy,0.354363
3,0.709266,0.630260,0.426421,0Gc6TVm52BwZD07Ki6tIvf,0.242801
4,0.763386,0.510344,0.486433,0IuslXpMROHdEPvSl1fTQK,0.313649


#### Get the 5 smallest distances and show the visuals for the top 2

In [ ]:
encoded_data['e_distance'].nsmallest(5)

0         5.960464e-08
137871    3.013010e-03
190202    3.036266e-03
69761     3.136666e-03
117514    3.136666e-03
Name: e_distance, dtype: float64

In [ ]:
print(df['track_name'][0])
print(df['track_name'][137871])
print(df['track_name'][190202])

C'est beau de faire un Show
Push Push
Los Angeles Is Burning


In [ ]:
df.iloc[[0,48095,38564]]


,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.61100,0.389,99373,0.910,0.0000,C#,0.346,-1.828,Major,0.0525,166.969,4/4,0.814
48095,Blues,Shovels & Rope,The Wire,6XYY2YLbbQ6oNaDTrk5DGU,41,0.00622,0.557,176890,0.554,0.0000,B,0.103,-6.506,Minor,0.0506,135.084,4/4,0.628
38564,Electronic,Moksi,Dream,2h4EfdqxQsoCEL7eKIdUGz,31,0.02050,0.829,170887,0.769,0.0663,B,0.152,-7.388,Major,0.0702,123.984,4/4,0.523


In [ ]:
df_T = df_scaled.T

In [ ]:
import plotly.express as px

fig = px.bar_polar(df_T, r = 0,hover_name=headers)
fig.show()

In [ ]:
fig = px.bar_polar(df_T, r = 48095, hover_name=headers)
fig.show()

## Build a Function for DS Unit 3 to use

In [41]:
df.iloc[[13918]]

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
13918,Dance,Britney Spears,Oops!...I Did It Again,6naxalmIoLFWR0siv8dnQQ,77,0.3,0.751,211160,0.834,0.000018,1,0.355,-5.444,0,0.0437,95.053,4,0.894


In [96]:
# test
song_at = [[77,0.3,0.751,211160,0.834,0.000018,0.355,-5.444,0.0437,95.053,0.894,1,0,4]]

In [92]:
### Read in the CSV ###
database = pd.read_csv('encoded_data.csv')

In [97]:
def predict(song_attributes, database):
  """
  Need to pass in a variable that is an array with the below song attributes 
  in order.
  [[popularity, acousticness, danceability, duration_ms, energy, instrumentalness,
  liveness, loudness, speechiness, tempo, valence, key, mode, time_signature]]
  """

  import pandas as pd
  import pickle

  df =  pd.DataFrame(song_attributes)

  ### Scale Input ###

  # import the pickled scaler model
  pickle_filename_1 = 'scaler_model.pkl'
  scaler_pkl = open(pickle_filename_1, 'rb')
  scaler = pickle.load(scaler_pkl) 

   # scale the input data
  input_scaled = scaler.transform(df)

  ### Encode Input ###

  # Loading the pickled autoencoder model
  pickle_filename = 'autoencoder_model.pkl'
    
  autoencoder_pkl = open(pickle_filename, 'rb')
  autoencoder = pickle.load(autoencoder_pkl)
  # encode the input data and set to variable
  input_x_y_z = (autoencoder.predict(input_scaled)[0][0], 
                 autoencoder.predict(input_scaled)[0][1],
                 autoencoder.predict(input_scaled)[0][2])

  ### Get distances ###
  def get_e_dist(my_df):
    from scipy.spatial import distance
    return distance.euclidean(my_df[[0,1,2]], input_x_y_z)

  database['e_distance'] = database.apply(get_e_dist, axis=1)

  five_smallest = database[['e_distance','track_id']].nsmallest(5, columns='e_distance', keep='all')

  five_closest = five_smallest['track_id']

  return five_closest.to_json()

In [98]:
test = predict(song_at, database)
test

'{"123296":"1sh3CLfGBEX28rdnIi827U","82894":"7r0lwexkZgNoTI6kb0skPB","210190":"69eD0IfAqEIUnKX63Npksy","107627":"2nrDcpSp46mVXsYtunWoen","105185":"1oXjlewRQDPoozHy3Cbwob"}'

In [99]:
df.iloc[[13918,123296,82894,210190,107627,105185]]

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
13918,Dance,Britney Spears,Oops!...I Did It Again,6naxalmIoLFWR0siv8dnQQ,77,0.300,0.7510,211160,0.83400,0.000018,1,0.3550,-5.444,0,0.0437,95.053,4,0.8940
123296,Classical,Giacomo Puccini,"Madame Butterfly, SC 74, Act 2: ""Coro a bocca ...",1sh3CLfGBEX28rdnIi827U,0,0.986,0.1830,194280,0.00352,0.957000,10,0.1290,-38.472,1,0.0424,71.523,3,0.0487
82894,Opera,Giacomo Puccini,Madama Butterfly / Act 2: Humming Chorus,7r0lwexkZgNoTI6kb0skPB,5,0.986,0.0909,195627,0.01180,0.963000,10,0.1060,-38.297,1,0.0416,64.640,3,0.0857
210190,World,Brian Eno,Discreet Music - Remastered 2004,69eD0IfAqEIUnKX63Npksy,37,0.978,0.0926,1894893,0.00839,0.939000,8,0.0599,-33.854,1,0.0443,69.020,1,0.0332
107627,Opera,Giacomo Puccini,Madame Butterfly (Highlights): Coro a bocca ch...,2nrDcpSp46mVXsYtunWoen,2,0.979,0.1550,183333,0.00510,0.913000,10,0.0752,-38.992,1,0.0444,83.727,3,0.0430
105185,Classical,Franz Liszt,"Via Crucis, S. 53: Station XIII. Jesus wird vo...",1oXjlewRQDPoozHy3Cbwob,12,0.987,0.4310,196840,0.00123,0.937000,10,0.1130,-46.985,1,0.0687,77.729,4,0.0417
